In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

%matplotlib tk
%load_ext autoreload

### Set style for plots

In [6]:
sns.set(style="ticks")
sns.set_style("darkgrid")

### Read data from excel

In [7]:
df = pd.read_excel('combat_losses.xlsx', engine='openpyxl')
df = df.set_index('date')

### Quick look at data

In [8]:
df.tail()

,personnel,tanks,APV,artillery systems,MLRS,anti-aircraft warfare systems,aircraft,helicopters,vehicles,boats / cutters,fuel tanks,UAV operational-tactical level,special equipment,mobile SRBM system
date,,,,,,,,,,,,,,
2022-03-27,16600,582,1664,294,93,52,121,127,1144,7,73,56,21,4
2022-03-28,17000,586,1694,302,95,54,123,127,1150,7,73,66,21,4
2022-03-29,17200,597,1710,303,96,54,127,129,1178,7,73,71,21,4
2022-03-30,17300,605,1723,305,96,54,131,131,1184,7,75,81,21,4
2022-03-31,17500,614,1735,311,96,54,135,131,1201,7,75,83,22,4


In [9]:
df.columns

Index(['personnel', 'tanks', 'APV', 'artillery systems', 'MLRS',
       'anti-aircraft warfare systems', 'aircraft', 'helicopters', 'vehicles',
       'boats / cutters', 'fuel tanks', 'UAV operational-tactical level',
       'special equipment', 'mobile SRBM system'],
      dtype='object')

In [10]:
df.describe()

,tanks,APV,artillery systems,MLRS,anti-aircraft warfare systems,aircraft,helicopters,vehicles,boats / cutters,fuel tanks,UAV operational-tactical level,special equipment,mobile SRBM system
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,374.083333,1202.083333,168.083333,59.666667,31.527778,70.861111,83.083333,668.916667,3.305556,60.166667,21.638889,7.111111,0.611111
std,168.000744,431.609769,95.524230,28.505639,18.060986,38.808065,42.194363,373.522872,1.954035,14.155262,25.531105,8.574973,1.419814
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,264.500000,943.500000,105.000000,50.000000,18.750000,37.500000,39.250000,407.750000,2.000000,60.000000,3.000000,0.000000,0.000000
50%,381.500000,1237.500000,145.000000,63.000000,34.000000,75.500000,88.000000,608.500000,3.000000,60.000000,7.500000,0.000000,0.000000
75%,511.000000,1561.500000,255.750000,80.000000,45.500000,99.500000,123.250000,1002.000000,3.250000,70.000000,36.750000,15.000000,0.000000
max,614.000000,1735.000000,311.000000,96.000000,54.000000,135.000000,131.000000,1201.000000,7.000000,75.000000,83.000000,22.000000,4.000000


### Define functions to handle data approximation
As we see, we have approximate data in personnel losses such as 'less than' or 'more than'.

To handle it we use average losses between 2 war days.

In [11]:
def handle_error(el):
    if str(el)[-1] == '+':
        return 1
    elif str(el)[-1] == '-':
        return -1
    
    return 0

def remove_sign(el):
    if str(el)[-1] in ['+', '-']:
        return int(str(el)[:-1])
    
    return int(el)

def find_error(df):
    n = len(df)

    for i in range(n):
        if df.iloc[i, 0] != remove_sign(df.iloc[i, 0]):
            df.iloc[i, 0] = remove_sign(df.iloc[i, 0]) + handle_error(df.iloc[i, 0]) * ((remove_sign(df.iloc[i+1, 0]) - remove_sign(df.iloc[i, 0])) // 2)


In [12]:
find_error(df)
df = df.fillna(0)
df = df.astype(int)
df.tail()

,personnel,tanks,APV,artillery systems,MLRS,anti-aircraft warfare systems,aircraft,helicopters,vehicles,boats / cutters,fuel tanks,UAV operational-tactical level,special equipment,mobile SRBM system
date,,,,,,,,,,,,,,
2022-03-27,16600,582,1664,294,93,52,121,127,1144,7,73,56,21,4
2022-03-28,17000,586,1694,302,95,54,123,127,1150,7,73,66,21,4
2022-03-29,17200,597,1710,303,96,54,127,129,1178,7,73,71,21,4
2022-03-30,17300,605,1723,305,96,54,131,131,1184,7,75,81,21,4
2022-03-31,17500,614,1735,311,96,54,135,131,1201,7,75,83,22,4


### Daily losses
Let's construct a data frame with daily, not total, losses.

In [13]:
daily_df = pd.DataFrame()

daily_df['date'] = df.index
daily_df = daily_df.set_index('date')
daily_df = df.diff().iloc[:, :].fillna(0).astype(int)

### Quick look at daily losses

In [14]:
daily_df.tail()

,personnel,tanks,APV,artillery systems,MLRS,anti-aircraft warfare systems,aircraft,helicopters,vehicles,boats / cutters,fuel tanks,UAV operational-tactical level,special equipment,mobile SRBM system
date,,,,,,,,,,,,,,
2022-03-27,200,7,24,1,2,1,4,0,13,0,0,0,2,2
2022-03-28,400,4,30,8,2,2,2,0,6,0,0,10,0,0
2022-03-29,200,11,16,1,1,0,4,2,28,0,0,5,0,0
2022-03-30,100,8,13,2,0,0,4,2,6,0,2,10,0,0
2022-03-31,200,9,12,6,0,0,4,0,17,0,0,2,1,0


In [15]:
daily_df.describe()

,personnel,tanks,APV,artillery systems,MLRS,anti-aircraft warfare systems,aircraft,helicopters,vehicles,boats / cutters,fuel tanks,UAV operational-tactical level,special equipment,mobile SRBM system
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,486.111111,17.055556,48.194444,8.638889,2.666667,1.500000,3.750000,3.638889,33.361111,0.194444,2.083333,2.305556,0.611111,0.111111
std,498.227015,14.006008,71.356978,10.621145,4.140393,1.732051,3.467193,4.574168,49.041762,0.524783,7.076621,3.214427,1.728381,0.464621
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,200.000000,8.750000,16.750000,2.000000,0.000000,0.000000,1.750000,0.000000,12.250000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,300.000000,13.500000,29.000000,5.500000,1.500000,1.000000,2.500000,2.000000,21.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,575.000000,18.000000,40.000000,11.000000,3.250000,2.000000,5.250000,4.250000,33.500000,0.000000,0.000000,3.000000,1.000000,0.000000
max,2200.000000,72.000000,410.000000,49.000000,17.000000,7.000000,16.000000,20.000000,261.000000,2.000000,30.000000,11.000000,10.000000,2.000000


### Plot data frames
Here we define a function to plot the whole data.

We should put a data frame, size, columns window and benchmark as parameters to tell whether it is Daily or Total df.

In [16]:
x_ticks = list(map(lambda el: str(el).split()[0], list(df.index)))

def draw(df, height, width, end, mark):
    fig, axs = plt.subplots(height, width)

    columns = iter(df.columns[:end])

    for i in range(3):
        for j in range(4):
            col = next(columns)
            axs[i, j].plot(x_ticks, df[col])
            axs[i, j].set_xticklabels(x_ticks, rotation=60)
            axs[i, j].set_title(f'{mark} War losses of {col}')
        
    for col, ax in zip(df.columns, axs.flat):
        ax.set(ylabel=col)

    fig.set_figheight(40)
    fig.set_figwidth(40)

In [17]:
draw(df, 3, 4, len(df)-1, 'Total')

<IPython.core.display.Javascript object>

In [18]:
draw(daily_df, 3, 4, len(daily_df)-1, 'Daily')

<IPython.core.display.Javascript object>

### Boxplot
Let's take a look at boxplots of daily losses.

As we see, the scale of features is different and it is quite hard to describe each feature on 1 plot.

In [19]:
fig, axs = plt.subplots(2, 2)

sns.boxplot(data=daily_df.iloc[:, :1], ax=axs[0, 0])
sns.boxplot(data=daily_df.iloc[:, [1, 3]], ax=axs[0, 1])
sns.boxplot(data=daily_df.iloc[:, [2, 8]], ax=axs[1, 0])
sns.boxplot(data=daily_df.iloc[:, 6:8], ax=axs[1, 1])

fig.set_figheight(10)
fig.set_figwidth(10)

<IPython.core.display.Javascript object>

### Heatmap
Let's take a look at correlation between different losses

In [20]:
sns.heatmap(df.iloc[:, :-2].corr())

<IPython.core.display.Javascript object>

<AxesSubplot:>

### War duration (in hours)

In [21]:
war_duration_hours = round((pd.to_datetime('today') - df.index[0]) / pd.Timedelta(hours=1), 2)

print(f'War is going on {len(df)} days ({war_duration_hours} hours)')

War is going on 36 days (856.01 hours)


### Feature creation
We create feature as range of real numbers instead of date to predict total losses till the given day.

In [22]:
df = df.dropna()

X_log = np.array([np.arange(1, len(df) + 1), np.log(np.arange(1, len(df) + 1))]).T
X = np.array([np.arange(1, len(df) + 1)]).T
y = df[['personnel']]

### Logarithmic Regression
Here we use obtained features to construct Logarithmic model for prediction of the total deaths of russian troops.

As we have few data, it is hard to evaluate our model right now.

In [23]:
log = LinearRegression().fit(X_log, y)

### Linear Regression
Let's apply linear model to our data as well.

In [24]:
lm = LinearRegression().fit(X, y)

### Polynomial Regression
Here it is polynomial regression of degree 3.

In [25]:
polynomial_regression = make_pipeline(PolynomialFeatures(degree=3, include_bias=False), LinearRegression())
polynomial_regression.fit(X, y)

Pipeline(steps=[('polynomialfeatures',
                 PolynomialFeatures(degree=3, include_bias=False)),
                ('linearregression', LinearRegression())])

### Prediction curve
Let's define some functions to handle predictions and plot it.

We will plot both prediction lines and scatter of real data to compare.

In [26]:
def predict_deaths(model, model_name, start, end):
    features = [i for i in range(start, end+1)]

    if model_name == 'Logarithmic Regression':
        X = np.array(list(zip(features, np.log(features))))
    else:
        X = [[i] for i in range(start, end+1)]

    y_hat = model.predict(X)
    y_hat[0] = max(0, y_hat[0])

    return features, y_hat

def predicted_curves(features, predicted, model_name):
    plt.figure(figsize=(10, 7))

    plt.plot(features[:min(len(y), len(features))], y[:min(len(y), len(features))], 'o', ms=4, label = 'Real Deaths')
    plt.plot(features, predicted, label = 'Predicted Deaths')

    plt.xlabel('day index')
    plt.ylabel('deaths')

    plt.title(f'Real vs Predicted Deaths using {model_name}')
    plt.legend()

### Models comparing
Comparing models highlighted above, we may observe that Logarithmic Regression is the best estimate of our population, because predictions looks more realistic. Of course, when we get enough data, we will be able to evaluate our models, prevent it from overfitting/underfitting and apply parameters tunning to increase some metrics.

In [27]:
num_days = 60

In [28]:
models = {'Logarithmic Regression': log, 'Linear Regression': lm,  'Polynomial Regression': polynomial_regression}

for model_name, model in models.items():
    features, predicted = predict_deaths(model, model_name, 1, num_days)
    predicted_curves(features, predicted, model_name)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Write obtained results in the csv file

In [29]:
data = list(zip(X.T[0], np.array(y)))
data = list(map(lambda el: [int(el[0]), int(el[1])], data))

features, predicted = predict_deaths(log, 'Logarithmic Regression', data[-1][0]+1, num_days)

predicted = list(map(int, predicted))
data.extend(list(map(list, list(zip(features, predicted)))))

with open('losses.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(['day index', 'deaths'])
    write.writerows(data)

## Conclusion
Russian troops should become more conscious.They will join to the part of sad statistics otherwise. Ukraine doesn't want to kill, but Ukraine wants to defend it's people. Get away of our country!

Русский военный корабль, иди нахуй
#### Glory to Ukraine!
#### Слава Україні!

## Data source
General Staff of the Armed Forces of Ukraine
 
https://www.facebook.com/GeneralStaff.ua

## Credits
© Mykhailo-Markiian Tsalyk

https://www.linkedin.com/in/markiian-tsalyk-193758224/